## Finetune PaliGemma for Object Detection 

#### CREATE PALIGEMMA STYLE DATASET

In [1]:
from pg_datasets import coco_to_xyxy, show_random_elements, string2list, resize_bbox, coco_to_xyxy, convert_to_detection_string

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset

xray_dataset = load_dataset('dmusingu/object-detection-chest-x-ray')

In [4]:
train_dataset = xray_dataset['train']
valid_dataset = xray_dataset['validation']
test_dataset = xray_dataset['test']

In [5]:
show_random_elements(train_dataset.remove_columns(['image']), num_examples=2)

,id,image_id,category_id,bbox,area,segmentation,iscrowd,license,height,width
0,641,602,Necklace,"[292, 1, 401, 122]",48922.0,[],0,1,1024,1024
1,731,688,Vascular Port,"[278, 280, 92.5, 115]",10637.5,[],0,1,1024,1024


In [6]:
train_dataset = train_dataset.map(lambda x: {'bbox': string2list(x['bbox'])})
valid_dataset = valid_dataset.map(lambda x: {'bbox': string2list(x['bbox'])})
test_dataset = test_dataset.map(lambda x: {'bbox': string2list(x['bbox'])})

In [7]:
train_dataset = train_dataset.map(resize_bbox)
valid_dataset = valid_dataset.map(resize_bbox)
test_dataset = test_dataset.map(resize_bbox)

In [8]:
train_dataset = train_dataset.map(coco_to_xyxy)
valid_dataset = valid_dataset.map(coco_to_xyxy)
test_dataset = test_dataset.map(coco_to_xyxy)

In [9]:
train_dataset = train_dataset.map(convert_to_detection_string)
valid_dataset = valid_dataset.map(convert_to_detection_string)
test_dataset = test_dataset.map(convert_to_detection_string)

In [10]:
print(train_dataset)

Dataset({
    features: ['image', 'id', 'image_id', 'category_id', 'bbox', 'area', 'segmentation', 'iscrowd', 'license', 'height', 'width', 'resized_bbox', 'xyxy', 'suffix'],
    num_rows: 736
})


In [11]:
# create a paligemma prefix
dataset_objects = list(set(train_dataset['category_id']))
dataset_objects

['Shoulder Endoprosthesis', 'Vascular Port', 'Necklace', 'ICD']

In [12]:
prefix = 'detect ' + "; ".join(dataset_objects)
prefix

'detect Shoulder Endoprosthesis; Vascular Port; Necklace; ICD'

In [13]:
train_prefix = [prefix] * len(train_dataset)
valid_prefix = [prefix] * len(valid_dataset)
test_prefix = [prefix] * len(test_dataset)

In [14]:
train_dataset = train_dataset.add_column('prefix', train_prefix)
valid_dataset = valid_dataset.add_column('prefix', valid_prefix)
test_dataset = test_dataset.add_column('prefix', test_prefix)

In [15]:
show_random_elements(train_dataset.remove_columns(['image']), num_examples=2)

,id,image_id,category_id,bbox,area,segmentation,iscrowd,license,height,width,resized_bbox,xyxy,suffix,prefix
0,279,265,Vascular Port,"[840.0, 555.0, 82.5, 112.5]",9281.25,[],0,1,1024,1024,"[183, 121, 18, 24]","[183, 121, 201, 145]",<loc0121><loc0183><loc0145><loc0201> Vascular Port,detect Shoulder Endoprosthesis; Vascular Port; Necklace; ICD
1,462,437,Necklace,"[333.0, 0.0, 347.5, 80.0]",27800.00,[],0,1,1024,1024,"[72, 0, 76, 17]","[72, 0, 148, 17]",<loc0000><loc0072><loc0017><loc0148> Necklace,detect Shoulder Endoprosthesis; Vascular Port; Necklace; ICD


### Pytorch Dataset and DataLoader

In [16]:
from pg_datasets import ObjectDetectionDataset
from torch.utils.data import DataLoader

In [17]:
pg_train_dataset = ObjectDetectionDataset(train_dataset)
pg_valid_dataset = ObjectDetectionDataset(valid_dataset)
pg_test_dataset = ObjectDetectionDataset(test_dataset)

In [20]:
pg_train_dataloader = DataLoader(pg_train_dataset, batch_size=2, shuffle=True, collate_fn=pg_train_dataset.collate_fn)
pg_valid_dataloader = DataLoader(pg_valid_dataset, batch_size=2, shuffle=False, collate_fn=pg_valid_dataset.collate_fn)
pg_test_dataloader = DataLoader(pg_test_dataset, batch_size=2, shuffle=False, collate_fn=pg_test_dataset.collate_fn)

In [ ]:
train_sample = next(iter(pg_train_dataloader))